# Import modules

In [1]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy


# Functions (helpers)

In [2]:
def read_csv(name, info=True, sep=','):
    df = pd.read_csv(f'../data/demographics/{name}.csv', sep=sep)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print('\n')
    print( '*' + '-' * 135 + '*')
    print('* ' + title)
    print( '*' + '-' * 135 + '*')

def print_subsection(title):
    print('\n')
    print('***** ' + title + ' *****')
    print( '*' + '-' * 135 + '*')

In [3]:
# filenames = [
#    'interleaved_incomplete', 'block_complete',
#    'block_complete_simple',
#    'block_complete_mixed',  'block_complete_mixed_2s','block_complete_mixed_2s_amb_heuristic',
#    'block_complete_mixed_2s_amb_final', 'incentivize', 'evoutcome'

# ]
# dem = {}
# for f in filenames:
#     if f == 'block_complete_mixed_2s_amb_final':
#         dem[f] = pd.read_csv('../data/demographics/' + f + '.csv', sep=';')
#     else: dem[f] = read_csv(f)

# # d = dem["block_incomplete"][dem['block_incomplete']['status'] == 'APPROVED']
# # d
# df = {}
# ids = pd.read_csv('../sub_ids.csv')
# ids['prolific_id'] = 'nothing'
# for f in filenames:
#     df = pd.read_csv('../data/csv/' + f + '.csv', header=None)
#     # df.loc[-1] = [str(i) for i in range(len(df.columns))]  # adding a row
#     # df.index = df.index + 1  # shifting index
#     # df.sort_index(inplace=True) 
#     for i in ids[ids.exp_name==f]['id']:
#         try:
#             prolific_id = df[df[1]==i][2].unique()[0]
#         # break
#             ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#         except:
#             try:
#                 prolific_id = df[df[1]==str(i)][2].unique()[0]
#                 ids.loc[ids.id==i, 'prolific_id'] = prolific_id
#             except:
#                 print(i, ' is missing...')
    
# # for i in ids['id']:
#     # df
# ids.to_csv('../sub_ids_with_prolific_id.csv', index=False)


In [4]:
# df = pd.read_csv('../sub_ids_with_prolific_id.csv')

# ids = df['prolific_id'].unique()
# exps = df['exp_name'].unique()
# df['age'] = np.nan
# df['gender'] = np.nan

# for f in exps:
#     sep = ','
#     if f == 'block_incomplete':
          # get prolific ids from demographics
#         ddf = pd.read_csv('../data/demographics/block_incomplete.csv')
#         ddf = ddf[ddf.status == 'APPROVED']
#         df.loc[df.exp_name=='block_incomplete', 'prolific_id'] = idx
#         ids = df['prolific_id'].unique()

#     if f == 'block_complete_mixed_2s_amb_final':
#         sep = ';'
#     print(f)
#     dem = pd.read_csv('../data/demographics/' + f + '.csv', sep=sep)
#     for i in ids:
#         try:
#             df.loc[df.prolific_id==i, 'gender'] = dem.loc[dem['participant_id'] == i, 'Sex'].values[0]
#             df.loc[df.prolific_id==i, 'age'] = dem.loc[dem['participant_id'] == i, 'age'].values[0]
#         except:
#             print(i, ' is missing...')
#             pass
# df.to_csv('../data/demographics/included_subjects.csv')

In [40]:


df = pd.read_csv('../data/demographics/included_subjects_without_revoked.csv')
#df.loc[(df['gender'].isna()) | (df['gender']=='CONSENT REVOKED'), 'gender'] = 'DATA_EXPIRED'
#df.to_csv('../data/demographics/included_subjects_without_revoked.csv')

#df[df['age'] == 'DATA_EXPIRED'] = np.random.choice('')
d = []
for e in df.exp_num.unique():
    if e == 10: continue

    gender = df[df.exp_num==e].gender.value_counts()
    age = df[(df.exp_num==e) & (df.age != 'DATA_EXPIRED')]
    exp_name = df[df.exp_num==e].exp_name.values[0]
    without = gender.drop(labels=['Male', 'Female'], inplace=False)
    other_gender = without.sum()
    
    print(gender)

    d.append(
        {   'exp_num': e,
            'exp_name': exp_name,
            'age_std': age.age.astype(float).std(),
            'age': age.age.astype(float).mean(),
            'prefer_not': other_gender,
            'M': gender['Male'],
            '%M': (gender['Male']/sum(gender)).round(2),
            'F': gender['Female'],
            '%F': (gender['Female']/sum(gender)).round(2),
            'N': len(df[df.exp_num==e])

    })

new_df = pd.DataFrame(d) 
new_df.N.sum()
# new_df.age.mean()
# new_df.age_std.mean()

new_df.F.sum()
new_df.M.sum()
new_df.prefer_not.sum()
307+331+35

Female          35
Male            34
DATA EXPIRED     4
DATA_EXPIRED     3
Name: gender, dtype: int64
Female          39
Male            26
DATA EXPIRED     6
Name: gender, dtype: int64
Male            41
Female          38
DATA_EXPIRED     2
DATA EXPIRED     2
Name: gender, dtype: int64
Male            48
Female          33
DATA EXPIRED     5
DATA_EXPIRED     2
Name: gender, dtype: int64
Male            38
Female          31
DATA EXPIRED     2
Name: gender, dtype: int64
Female          32
Male            32
DATA EXPIRED     2
Name: gender, dtype: int64
Female          36
Male            32
DATA_EXPIRED     3
Name: gender, dtype: int64
Female          38
Male            33
DATA EXPIRED     2
Name: gender, dtype: int64
Male            47
Female          25
DATA_EXPIRED     2
Name: gender, dtype: int64


673

307

331

35

673